In [1]:
import os

os.environ['https_proxy'] = 'http://127.0.0.1:15777'
os.environ['http_proxy'] = 'http://127.0.0.1:15777'

In [20]:
import hashlib
import numpy as np

def encode_node_id(node_id):
    node_id_str = str(node_id)
    
    hash_functions = [
        hashlib.md5,
        hashlib.sha1,
        hashlib.sha256,
        hashlib.sha512
        ]
    
    hash_values = []
    
    for hash_func in hash_functions:
        hash_object = hash_func(node_id_str.encode())
        hash_bytes = hash_object.digest()
        print(hash_bytes)
        hash_ints = [b for b in hash_bytes]
        print(len(hash_ints))
        hash_values.extend(hash_ints)
    
    vector = np.array(hash_values) / 255.0
    
    return vector

node_id = 121
encoded_vector = encode_node_id(node_id)
print(encoded_vector)
print(f"Vector length: {len(encoded_vector)}")

b'LV\xffL\xe4\xaa\xf9W:\xa5\xdf\xf9\x13\xdf\x99z'
16
b'\x8b\xd7\x95L@\xc1\xe5\x9a\x90\x0fq\xea:&g2`\x99\x15\xb1'
20
b'\x89\xaa\x1eX\x00#r-\xb6vF\xe8\x14\x9e\xb2F\xc7H\xe1\x80\xe3J\x1c\xf6y\xab\x0bA\xa4\x16\xd9\x04'
32
b'67\t\xc1\xbe\xd1\x00>\xf6\x9eXl\xbc\x9dr\x84\xf1\x91g>3\x96\x01\xd9\x00\x9a>\x1925\xfb\xcf+z\x14\x84l(9\xafA\x8a\xb5.\n\x89\xd6|\xb7\xb0\x8f\x9b\xc1_h\x8a\x85\xb4\x91\xb8\xff\t\x97\x1f'
64
[0.29803922 0.3372549  1.         0.29803922 0.89411765 0.66666667
 0.97647059 0.34117647 0.22745098 0.64705882 0.8745098  0.97647059
 0.0745098  0.8745098  0.6        0.47843137 0.54509804 0.84313725
 0.58431373 0.29803922 0.25098039 0.75686275 0.89803922 0.60392157
 0.56470588 0.05882353 0.44313725 0.91764706 0.22745098 0.14901961
 0.40392157 0.19607843 0.37647059 0.6        0.08235294 0.69411765
 0.5372549  0.66666667 0.11764706 0.34509804 0.         0.1372549
 0.44705882 0.17647059 0.71372549 0.4627451  0.2745098  0.90980392
 0.07843137 0.61960784 0.69803922 0.2745098  0.78039216 

In [9]:
y_pred

array([12,  8,  1, ...,  9,  6, 17])

In [28]:
from ogb.nodeproppred import Evaluator
import torch


def evaluate(pred, label):
    input_dict = {"y_true": label, "y_pred": pred}
    return Evaluator(name='ogbn-arxiv').eval(input_dict)

x = torch.randint(0, 40, (100000, 1))
y = torch.randint(0, 40, (100000, 1))
print(evaluate(x, y))

{'acc': 0.02531}


In [2]:
from ogb.nodeproppred import PygNodePropPredDataset
import pandas as pd
from dataset.dataloader import *
from dataset.embedding import *
from torch_geometric.loader import NeighborLoader, DataLoader
import torch_geometric.transforms as T

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [30]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', root='./data')
split_idx = dataset.get_idx_split()
graph = dataset[0]
data = {
        'train_idx': split_idx['train'], # 90941
        'valid_idx': split_idx['valid'], # 29799
        'test_idx': split_idx['test'],   # 48603
        'graph': graph
        }
graph

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])

In [28]:
import torch
import numpy as np
a = torch.tensor([1,2,3,4,5,6,7,8,9,10, 1, 1])
d = {}
for i, x in enumerate(a):
    d[x] = i
d

{tensor(1): 0,
 tensor(2): 1,
 tensor(3): 2,
 tensor(4): 3,
 tensor(5): 4,
 tensor(6): 5,
 tensor(7): 6,
 tensor(8): 7,
 tensor(9): 8,
 tensor(10): 9,
 tensor(1): 10,
 tensor(1): 11}

In [ ]:
pt = torch.load()

In [5]:
def naive_search(a, b=None):
    for i in range(graph.edge_index.shape[1]):
        if b is not None:
            if graph.edge_index[0][i] == a and graph.edge_index[1][i] == b:
                print(i)
                return
        else:
            if graph.edge_index[0][i] == a:
                print(i, graph.edge_index[1][i])
        
naive_search(13091)


7499 tensor(42537)
7500 tensor(124512)


In [39]:
batch.edge_index

EdgeIndex([[1024, 1025, 1026,  ..., 3291, 3292, 3293],
           [   1,    1,    2,  ..., 2459, 2461, 2462]],
          sparse_size=(3294, 3294), nnz=2374, sort_order=col)

In [40]:
import random
from typing import List, Tuple


class BiasedRandomWalker:

    def __init__(self, db, p: float = 1.2, q: float = 2.0):
        self.db = db
        self.ret_p = p
        self.io_q = q

        self.connected_nodes = self._get_connected_nodes()

    def _get_connected_nodes(self):

        txn = self.db.CreateReadTxn()
        vit = txn.GetVertexIterator()

        connected_nodes = []
        while vit.IsValid():
            if vit.GetNumOutEdges()[0] > 0:
                connected_nodes.append(vit.GetId())
            vit.Next()

        txn.Commit()
        return connected_nodes

    def _normalize(self, weights):
        tot = sum(weights)
        return [p / tot for p in weights]

    def get_probs_uniform(self, txn, vit) -> Tuple[List[int], List[float]]:
        nexts = vit.ListDstVids()[0]
        probs = [1 / len(nexts)] * len(nexts)
        return nexts, probs

    def get_probs_biased(self, txn, vit, prev: int) -> Tuple[List[int], List[float]]:
        curr_nbrs = vit.ListDstVids()[0]

        nexts = []
        unnormalized_probs = []
        for next in curr_nbrs:
            nexts.append(next)
            if next == prev:
                unnormalized_probs.append(1 / self.ret_p)
            elif txn.GetVertexIterator(next).HasEdge(prev):
                unnormalized_probs.append(1)
            else:
                unnormalized_probs.append(1 / self.io_q)

        probs = self._normalize(unnormalized_probs)
        return nexts, probs

    def walk(self, start: int, length: int) -> List[int]:

        txn = self.db.CreateReadTxn()
        vit = txn.GetVertexIterator(start)

        trace = [vit.GetId()]
        current_len = 1
        
        prev = None
        
        while current_len < length:
            if prev is None:
                nexts, probs = self.get_probs_uniform(txn, vit)
            else:
                nexts, probs = self.get_probs_biased(txn, vit, prev)

            target = random.choices(nexts, probs)[0]
            trace.append(target)

            vit.Goto(vid=target, nearest=False)
            current_len += 1

        txn.Commit()
        return trace


ImportError: cannot import name 'random_walk' from 'torch_geometric.utils' (/home/loping151/anaconda3/envs/dm/lib/python3.10/site-packages/torch_geometric/utils/__init__.py)